#### Filter the gene set based on the output from script `Step_01_remove_non_redundant_gene_set.R`

In [1]:
library(tidyverse)
library(pheatmap)
library(fgsea)

── Attaching core tidyverse packages ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


Load the original gene set file -- we will need to format our subsetted gene set according to this 

In [2]:
pathways <- gmtPathways("c5.go.bp.v2023.2.Hs.symbols.gmt")

Load the gene sets from the `Step_01`

In [3]:
distance_matrix <- readRDS("fgsea_gene_set_distance_matrix.rds")

In [4]:
dim(distance_matrix)

[1] 7647 7647

In [5]:
# run hierarchical clustering
hc <- hclust(as.dist(distance_matrix), method = "complete")

In [9]:
# define clusters as those whose dissimilarity is 70% or less
# the higher this value (close to 1), the fewer the clusters since h = 1 means everything that has a dissimilarity of 1 or less
# (which is everything since the Jaccard values are from 0 to 1) is one cluster
# the lower the value, the more clusters there are 0, so if h = 0, then every gene set is its own cluster
dissimilarity_threshold = 0.7
clusters <- cutree(hc, h = 0.7)

In [10]:
GO_term_cluster_assignments <- as.data.frame(clusters)

In [11]:
# number of clusters
num_clusters <- length(unique(GO_term_cluster_assignments$clusters))
num_clusters

[1] 3913

In [8]:
write.csv(GO_term_cluster_assignments, file = paste0("fgseapy_at_", dissimilarity_threshold, "_dissimilarity_GO_BP_terms.csv"))

In [9]:
# cluster names are strings 
cluster_names = as.character(seq(1, num_clusters))

Randomly sample one gene set per cluster

In [10]:
# Initialize an empty named character vector
representative_GO_term_per_cluster <- character(length(cluster_names))
names(representative_GO_term_per_cluster) <- cluster_names

# Loop to fill the vector with representative GO terms
for (cluster_name in cluster_names) {
    representative_GO_term_per_cluster[cluster_name] <- sample(names(clusters[clusters == cluster_name]), 1)
}

The original .gmt file is a list, we will just subset it to those represented GO terms

In [11]:
class(pathways)

[1] "list"

In [12]:
filtered_pathways <- pathways[names(pathways) %in% representative_GO_term_per_cluster]

In [13]:
head(filtered_pathways)

$GOBP_10_FORMYLTETRAHYDROFOLATE_METABOLIC_PROCESS
[1] "AASDHPPT" "ALDH1L1"  "ALDH1L2"  "MTHFD1"   "MTHFD1L"  "MTHFD2L" 

$GOBP_2_OXOGLUTARATE_METABOLIC_PROCESS
 [1] "AADAT"  "ADHFE1" "D2HGDH" "DLST"   "GOT1"   "GOT2"   "GPT2"   "IDH1"  
 [9] "IDH2"   "KYAT3"  "L2HGDH" "MRPS36" "OGDH"   "OGDHL"  "PHYH"   "TAT"   

$GOBP_3_PHOSPHOADENOSINE_5_PHOSPHOSULFATE_METABOLIC_PROCESS
 [1] "ABHD14B" "BPNT1"   "ENPP1"   "PAPSS1"  "PAPSS2"  "SULT1A1" "SULT1A2"
 [8] "SULT1A3" "SULT1A4" "SULT1B1" "SULT1C3" "SULT1C4" "SULT1E1" "SULT2A1"
[15] "SULT2B1" "TPST1"   "TPST2"  

$GOBP_3_UTR_MEDIATED_MRNA_DESTABILIZATION
 [1] "CPEB3"   "DHX36"   "DND1"    "HNRNPD"  "KHSRP"   "MOV10"   "PLEKHN1"
 [8] "PUM1"    "PUM2"    "RBM24"   "RC3H1"   "TARDBP"  "TRIM71"  "UPF1"   
[15] "ZC3H12A" "ZC3H12D" "ZFP36"   "ZFP36L1" "ZFP36L2"

$GOBP_3_UTR_MEDIATED_MRNA_STABILIZATION
 [1] "ANGEL2"   "BOLL"     "DAZ1"     "DAZ2"     "DAZ3"     "DAZ4"    
 [7] "DAZL"     "ELAVL1"   "ELAVL4"   "HNRNPA0"  "HNRNPC"   "LARP4B"  
[13] "MAPK14"   "MAPKAPK2" "METTL16"  "MYD88"    "RBM10"    "RBM24"   
[19] "RBM47"    "TARDBP"   "TENT4A"   "TENT4B"   "TIRAP"    "YBX3"    
[25] "ZFP36"   

$GOBP_4FE_4S_CLUSTER_ASSEMBLY
[1] "FDX2"  "FXN"   "ISCU"  "LYRM4" "NFS1"

Write this in the gmt format

In [14]:
# specify output and open a file connectio
output_file <- "filtered_GO_Hs_symbols.gmt"
file_conn <- file(output_file, "w")

# loop through all of the GO terms in the filtered set
for (go_term in names(filtered_pathways)) {
  # fetch the associated genes for this GO term
  genes <- filtered_pathways[[go_term]]
  
  # write to the file in GMT format: GO term, description (use GO term as placeholder), and genes
  line <- paste(go_term, go_term, paste(genes, collapse = "\t"), sep = "\t")
  
  # write the file to the file
  writeLines(line, file_conn)
}

# close the file connection
close(file_conn)

cat("File written to", output_file, "\n")

File written to filtered_GO_Hs_symbols.gmt 


In [15]:
reloaded_filtered_pathways <- gmtPathways(output_file)

In [16]:
head(reloaded_filtered_pathways)

$GOBP_10_FORMYLTETRAHYDROFOLATE_METABOLIC_PROCESS
[1] "AASDHPPT" "ALDH1L1"  "ALDH1L2"  "MTHFD1"   "MTHFD1L"  "MTHFD2L" 

$GOBP_2_OXOGLUTARATE_METABOLIC_PROCESS
 [1] "AADAT"  "ADHFE1" "D2HGDH" "DLST"   "GOT1"   "GOT2"   "GPT2"   "IDH1"  
 [9] "IDH2"   "KYAT3"  "L2HGDH" "MRPS36" "OGDH"   "OGDHL"  "PHYH"   "TAT"   

$GOBP_3_PHOSPHOADENOSINE_5_PHOSPHOSULFATE_METABOLIC_PROCESS
 [1] "ABHD14B" "BPNT1"   "ENPP1"   "PAPSS1"  "PAPSS2"  "SULT1A1" "SULT1A2"
 [8] "SULT1A3" "SULT1A4" "SULT1B1" "SULT1C3" "SULT1C4" "SULT1E1" "SULT2A1"
[15] "SULT2B1" "TPST1"   "TPST2"  

$GOBP_3_UTR_MEDIATED_MRNA_DESTABILIZATION
 [1] "CPEB3"   "DHX36"   "DND1"    "HNRNPD"  "KHSRP"   "MOV10"   "PLEKHN1"
 [8] "PUM1"    "PUM2"    "RBM24"   "RC3H1"   "TARDBP"  "TRIM71"  "UPF1"   
[15] "ZC3H12A" "ZC3H12D" "ZFP36"   "ZFP36L1" "ZFP36L2"

$GOBP_3_UTR_MEDIATED_MRNA_STABILIZATION
 [1] "ANGEL2"   "BOLL"     "DAZ1"     "DAZ2"     "DAZ3"     "DAZ4"    
 [7] "DAZL"     "ELAVL1"   "ELAVL4"   "HNRNPA0"  "HNRNPC"   "LARP4B"  
[13] "MAPK14"   "MAPKAPK2" "METTL16"  "MYD88"    "RBM10"    "RBM24"   
[19] "RBM47"    "TARDBP"   "TENT4A"   "TENT4B"   "TIRAP"    "YBX3"    
[25] "ZFP36"   

$GOBP_4FE_4S_CLUSTER_ASSEMBLY
[1] "FDX2"  "FXN"   "ISCU"  "LYRM4" "NFS1"

The filtered set is now compatible with fgseapy and can be properly loaded